In [1]:
import json
import re
from gensim.models import Word2Vec
import csv
import pandas as pd
from sklearn.cluster import DBSCAN

In [2]:
import snowballstemmer

In [3]:
f = open('digital_trajectoriesraject_public_keyword.json')
digital_trajectoriesraject_public_keyword = json.load(f)

f.close()

In [4]:
f = open('documents1.json')
documents = json.load(f)

f.close()

https://python-school.ru/blog/word2vec-with-examples-in-gensim/

## digital_trajectoriesraject_public_keyword

In [5]:
text = ""
for doc in documents: text += doc['text']

In [6]:
sentences = re.split(r'\.+\s', text.lower())


In [7]:
sentences[:3]

['коммерциализация результатов научных исследований и разработок представляет собой процесс вовлечения их в экономический (коммерческий) оборот в различных сегментах национального и глобального рынков',
 'актуальность данной дисциплины обусловлена становлением на путь модернизации экономики страны',
 'в сложившейся ситуации резко возрастает необходимость оценки и обоснования экономической эффективности проектов, ориентированных на выпуск высокотехнологичной продукции и продвижение новых технологий']

In [8]:
sentences_word_by_word = []
for i in sentences:
    i = re.sub(r'[^а-яёa-z -+.\/0-9]', '', i)
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    sentences_word_by_word.append(i.split(' '))

In [10]:
'''w2v_model = Word2Vec(
    min_count=1,
    window=2,
    negative=10,
    alpha=0.03,
    min_alpha=0.0007,
    sample=6e-5,
    sg=1)
'''

'w2v_model = Word2Vec(\n    min_count=1,\n    window=2,\n    negative=10,\n    alpha=0.03,\n    min_alpha=0.0007,\n    sample=6e-5,\n    sg=1)\n'

In [11]:
#w2v_model.build_vocab(sentences_word_by_word)

In [12]:
#w2v_model.train(sentences_word_by_word, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [13]:
#w2v_model.init_sims(replace=True)


In [14]:
#w2v_model.wv.save_word2vec_format(fname='w2v.txt', fvocab='w2v_vocab.txt')

In [15]:
#w2v_model.save('w2v_model.model')

In [16]:
w2v_model = Word2Vec.load('w2v_model.model')

In [17]:
arr = []
for i in digital_trajectoriesraject_public_keyword:
    i = i['value']
    i = re.sub('[^а-яёa-z -+.\/0-9]', '', i.lower())
    i = re.sub(r'\xa0', ' ', i)
    i = re.sub(r'\s+', ' ', i)
    arr.append(i)
    
digital_trajectoriesraject_public_keyword = list(set([] + arr))

len(digital_trajectoriesraject_public_keyword), digital_trajectoriesraject_public_keyword[:5]

(21727,
 ['биологических объектов',
  'основы технологии обработки материалов',
  'хх века',
  'алгоритмов теории управления',
  'изучение устройства'])

In [83]:
X = []
dictionary = {}
notInVocab = []

for phrase in digital_trajectoriesraject_public_keyword:
    vec = 0
    flag = 0
    #print(phrase)
    for word in phrase.split(' '):
        if not word in w2v_model.wv.key_to_index.keys():
            flag = 1
            notInVocab.append(word)
            break
        vec += w2v_model.wv[word]
    if flag:
        continue
    string = " ".join(str(v) for v in vec)
    if string in dictionary.keys():
        dictionary[string].append(phrase)
    else:
        dictionary[string] = [phrase]
    X.append(vec)
    
X[0]

array([ 0.11368366,  0.1534835 ,  0.28261477,  0.14253736, -0.03377892,
       -0.1325148 ,  0.01215758,  0.16479501,  0.0185507 , -0.16854782,
       -0.03401543, -0.04552145, -0.15744153,  0.02028718, -0.2915712 ,
       -0.0037047 ,  0.16905883, -0.35075518,  0.07604432, -0.29700595,
       -0.01102103,  0.32449514,  0.018736  ,  0.03866997, -0.02018026,
       -0.19745213, -0.12796786,  0.1485452 ,  0.09864835,  0.1459948 ,
        0.01216129,  0.11927241,  0.2285976 , -0.32617205, -0.00337032,
        0.17040387, -0.25191242, -0.1643639 ,  0.04505595, -0.2966292 ,
       -0.06749343,  0.12338302, -0.26707768, -0.14507507, -0.09573202,
       -0.07334553, -0.11509916,  0.34034172,  0.19504982,  0.08144306,
       -0.05076409,  0.04213373,  0.0400576 ,  0.05362323,  0.35313648,
        0.22733079,  0.22153682, -0.02329143, -0.04966372, -0.17534766,
       -0.04017783,  0.09120797, -0.29517448,  0.10454089,  0.06102083,
       -0.05846421,  0.16219747, -0.1454515 , -0.13897488,  0.26

In [19]:
for i in list(dictionary.keys())[:1]:
    print(i, dictionary[i], '\n\n')

0.11368366 0.1534835 0.28261477 0.14253736 -0.03377892 -0.1325148 0.012157582 0.16479501 0.018550698 -0.16854782 -0.03401543 -0.045521446 -0.15744153 0.020287178 -0.2915712 -0.0037047016 0.16905883 -0.35075518 0.07604432 -0.29700595 -0.0110210255 0.32449514 0.018736005 0.038669974 -0.02018026 -0.19745213 -0.12796786 0.1485452 0.098648354 0.1459948 0.012161292 0.11927241 0.2285976 -0.32617205 -0.0033703204 0.17040387 -0.25191242 -0.1643639 0.04505595 -0.2966292 -0.06749343 0.12338302 -0.26707768 -0.14507507 -0.09573202 -0.07334553 -0.11509916 0.34034172 0.19504982 0.08144306 -0.05076409 0.042133734 0.040057596 0.05362323 0.35313648 0.22733079 0.22153682 -0.023291431 -0.049663715 -0.17534766 -0.040177833 0.09120797 -0.29517448 0.10454089 0.06102083 -0.058464214 0.16219747 -0.1454515 -0.13897488 0.2636006 -0.021610744 0.024455696 0.08051905 0.08795194 -0.20381187 -0.36550638 -0.20061266 -0.28204295 -0.023007486 -0.1813288 -0.2322516 0.08025633 0.0073941033 0.13465896 0.13185349 -0.3330456

### DBSCAN

In [78]:
clustering = DBSCAN(eps=0.01, min_samples=2, algorithm='kd_tree').fit(X)


In [79]:
clustering.labels_

array([-1, -1, -1, ..., -1, -1, -1])

In [80]:
for i in range(-1, 20):
    print(list(clustering.labels_).count(i))

21557
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


In [81]:
number_of_clusters = clustering.labels_.max()
synonyms_array = []
size = len(X)
labels = list(clustering.labels_)

for i in range(number_of_clusters):
    arr = []
    start_index = 0
    while(1):
        if (i not in labels[start_index:]):
            break
        next_index = labels[start_index:].index(i)
        key = " ".join(str(v) for v in X[next_index])
        arr.extend(dictionary[key])
        start_index += next_index + 1
    synonyms_array.append(arr)
            

In [82]:
for i in range(len(synonyms_array)):
    print(i)
    for j in synonyms_array[i]:
        print(j)
    print()

0
устройствах свч
свч устройствах
значение языка с++

1
tcad synopsys
synopsys tcad
современное развитие

2
систем автономных
автономных систем
биологических процессов

3
методов организации
организации методов
изучению основ технологии

4
работы получение навыков
обеспечению защиты информации

5
систем поддержки
поддержки систем
основания типологии культур

6
основы формирования технологии
принцип действия двигателя

7
напряжения источника
источника напряжения
электронные устройства

8
применения методов
методов применения
информатизации управления инновациями

9
систем информационных
информационных систем
содержание эссе

10
объектов проектирования
проектирования объектов
обработка

11
представления моделей
моделей представления
знания маршрутов проектирования

12
ответа варианты
варианты ответа
обработки результатов исследований

13
алгоритмов оптимизации
оптимизации алгоритмов
производства цс

14
работы навыков
навыков работы
сертификации систем

15
работы формирование навыков
проц

In [75]:
for i in list(dictionary.keys()):
    if len(dictionary[i]) > 1:
        print(dictionary[i], '\n')

['устройствах свч', 'свч устройствах'] 

['tcad synopsys', 'synopsys tcad'] 

['систем автономных', 'автономных систем'] 

['методов организации', 'организации методов'] 

['систем поддержки', 'поддержки систем'] 

['напряжения источника', 'источника напряжения'] 

['применения методов', 'методов применения'] 

['систем информационных', 'информационных систем'] 

['объектов проектирования', 'проектирования объектов'] 

['представления моделей', 'моделей представления'] 

['ответа варианты', 'варианты ответа'] 

['алгоритмов оптимизации', 'оптимизации алгоритмов'] 

['работы навыков', 'навыков работы'] 

['компонентов информационных', 'информационных компонентов'] 

['различных лазеров', 'лазеров различных'] 

['классификации систем', 'систем классификации'] 

['излучения рентгеновского', 'рентгеновского излучения'] 

['свч устройств', 'устройств свч'] 

['случайных дискретных сигналов', 'дискретных случайных сигналов'] 

['проектирования средств', 'средств проектирования'] 

['систем м

### Мусор


In [18]:
len(X)

21627

In [52]:
len(notInVocab), notInVocab

(100,
 ['а.',
  'а.',
  'им.',
  'expressg.',
  'b.',
  'б.',
  'n.',
  'а.',
  'r.',
  '3g.',
  'м.д.с.',
  'а.',
  'вт.',
  'microstructured',
  'и.',
  'p.',
  'б.',
  'а.',
  'фишер',
  'э.',
  'lm.',
  'ч.',
  'ф.',
  'в2с.',
  'а.к.',
  'а.',
  'райт',
  'а.',
  'а.',
  'processor',
  'b.',
  'т.',
  'а.',
  'в.и.',
  'б.',
  'б.',
  'т.',
  'м.д.с.',
  'т.ч.',
  'f1(x',
  'а.',
  '5s.',
  'http(s',
  'maec',
  'ком.',
  'f1(x',
  'а.',
  'б.',
  '3d.',
  'languages',
  'а.',
  'geometry',
  'asi',
  'им.',
  'd.',
  'д.и.',
  'к.',
  'м.ю.',
  'а.',
  'l.',
  'ф.и.о.',
  'тыс.',
  'i.',
  'д.и.',
  'и.в.',
  'д.и.',
  'а.',
  'а.',
  'г.п.',
  'криптопроколов',
  'т.ч.',
  'а.',
  'прожект',
  'pow',
  '4d.',
  'б.',
  '#',
  'у.',
  'им.',
  'б.',
  'им.',
  'вс.',
  'c(x',
  'в2b.',
  'исон',
  'mdd',
  'f(x)x8+x6+x3',
  'f.',
  'а.',
  'а.',
  'а.',
  'kellogg',
  'e.',
  'ленина',
  'а.',
  'б.',
  'а.',
  'а.',
  'b.',
  'вс.'])